## fast snake

### load some libraries
and outline the structure of the actual battlesnake board structure

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
from tqdm import tqdm

# structure of a board_state
test_obs = {'board': {'height': 15, 'width': 15,
                      'snakes': [{'id': 'agent_0',
                                  'name': 'agent_0',
                                  'latency': '0',
                                  'health': 4,
                                  'body': [{'x': 8, 'y': 3},
                                           {'x': 7, 'y': 3},
                                           {'x': 7, 'y': 2}],
                                  'head': {'x': 8, 'y': 3},
                                  'length': 3,
                                  'shout': '', 
                                  'squad': '',
                                  'customizations': {'color': '#00FF00',
                                                     'head': '',
                                                     'tail': ''}}],
                      'food': [{'x': 13, 'y': 13},
                               {'x': 12, 'y': 10}],
                      'hazards': []}}


### very simple snake
define a very simple snake as a proof of concept

In [2]:

class simple_snake():
    # can add other functions, but needs to have 'move' and the property 'name' at very least
    # this snake won't run into obstacles, but nothing more than that.
    
    def __init__(self, name):
        self.name = name
        self.health = 100
        
    def move(self, board_state: dict) -> int:
        
        # open up the board_state and select a move that doesn't instantly kill the snake

        # the actual engine has the whole dictionary nested in another dictionary and needs the next line uncommented: (I think?)
        # board_state = board_state['board']
        
        # ---------------- obstacle array -----------------
        temp_array = np.zeros((board_state['height'], board_state['width'])) # not sure order is right
        # fill array with bodies of all snakes
        for _snake in board_state['snakes']:
            for _body_segment in _snake['body']:
                # this removes out of bounds 
                tempx = np.clip(_body_segment['x'],0,board_state['width']-1)
                tempy = np.clip(_body_segment['y'],0,board_state['height']-1)
                temp_array[tempx][tempy] = 1

        # get all indexes
        _temp_indexes = np.array(np.nonzero(temp_array)).T
        _temp_obs_dict = list()
        for _row in _temp_indexes:
            _temp_obs_dict.append({'x':_row[0], 'y':_row[1]})

        # add board edges to _temp_obs_dict
        for _width in range(board_state['width']):
            _temp_obs_dict.append({'x':_width, 'y':-1})
            _temp_obs_dict.append({'x':_width, 'y':board_state['height']})
        for _height in range(board_state['height']):
            _temp_obs_dict.append({'x':-1, 'y':_height})
            _temp_obs_dict.append({'x':board_state['width'], 'y':_height})

        # ------- create list of possible moves ----------
        for _snake in board_state['snakes']:
            if _snake['name'] == self.name:
                head_position = _snake['head'] # hold onto current location

        # these are 4 places this snake can move to right now.
        possible_moves = [{'x':head_position['x'],'y':head_position['y']+1},
                          {'x':head_position['x'],'y':head_position['y']-1},
                          {'x':head_position['x']-1,'y':head_position['y']},
                          {'x':head_position['x']+1,'y':head_position['y']}]
        
        # -------- determine which of the possible moves are safe --------
        safe_moves = list()
        for idx, _move in enumerate(possible_moves):
            if _move not in _temp_obs_dict:
                safe_moves.append(idx)
        
        # --------- pick a random safe move ---------
        if len(safe_moves)>0:
            return np.random.choice(safe_moves)
        
        else: 
            # ------- try to move to a tail of a snake --------
            # (this was an afterthought)
            for idx, _move in enumerate(possible_moves):
                for _snake in board_state['snakes']:
                    if _snake['body'][-1] == _move:
                        return idx 
            
            # ----- this snake is going to crash, but it needs to return some direction ------
            return np.random.choice([0,1,2,3])



### my game board
I have removed some unneeded keys, mostly kept the same

In [3]:
# the board_state i use (has reduced keys)

my_example_board_state = {'height': 15, 'width': 15,
 'snakes': [{'name': 'tom', 'health': 100,
             'head': {'x': 1, 'y': 0},
             'body': [{'x': 1, 'y': 1},
                      {'x': 1, 'y': 1},
                      {'x': 1, 'y': 1}],
             'length': 3},
            {'name': 'molly', 'health': 100,
             'head': {'x': 1, 'y': 14},
             'body': [{'x': 1, 'y': 14},
                      {'x': 1, 'y': 14},
                      {'x': 1, 'y': 14}],
             'length': 3}],
 'food': [{'x': 0, 'y': 0},
          {'x': 2, 'y': 2},
          {'x': 9, 'y': 9},
          {'x': 6, 'y': 6},
          {'x': 5, 'y': 5}]}


### rules
this seems to be close to the actual game

In [4]:


rules = {'starting_length': 3, # for constrictor, just set this to something huge.
         'game_mode': 'duel', # or 'solo'
         'food_max': 20, # 
         'food_rate': 0.15, # chance that a new food item is added
         'food_min': 1
        }

### the game engine
(lot of room for improvement in this code, but it works)

In [14]:
class game_engine():
    
    def init_on_game_state(self, game_state: dict, rules: dict):
        self.width = game_state['width']
        self.height = game_state['height']
        self.snakes = list()
        for _snake in game_state['snakes']:
            self.snakes.append(simple_snake(_snake['name']))
        self.rules = rules
        self.board_state = game_state
        self.history = list()
        
    def initialize(self,
                   board: tuple[int,int],
                   snakes: list[simple_snake],
                   rules: dict,
                  ):
        
        self.width = board[0]
        self.height = board[1]
        self.snakes = snakes # list of classes, 1 per snake
        self.rules = rules
        self.rules['starting_points'] = [{'x':1,'y':1},
                                         {'x':1,'y':self.height-2},
                                         {'x':self.width-1,'y':1},
                                         {'x':self.width-1,'y':self.height-2}]
        
        # fill in the board state at initialization
        self.board_state = dict()
        self.board_state['height'] = self.height
        self.board_state['width'] = self.width
        self.board_state['snakes'] = list()
        self.history = list() # full game history (as stored in the game states)
        
        # fill with snakes
        for idx, _snake in enumerate(snakes):
            temp_dict = {}
            temp_dict['name'] = _snake.name
            temp_dict['health'] = 100
            
            # internal value 'age' is used to track which body segments to pop
            temp_dict['head'] = {'x':self.rules['starting_points'][idx]['x'],
                                 'y':self.rules['starting_points'][idx]['y']}
            
            temp_dict['body'] = list()
            for _ in range(self.rules['starting_length']):
                temp_dict['body'].append({'x':self.rules['starting_points'][idx]['x'],
                                          'y':self.rules['starting_points'][idx]['y']})

            temp_dict['length'] = self.rules['starting_length']
            temp_dict['desired'] = None

            # add snake dictionary
            self.board_state['snakes'].append(temp_dict)
            
        # add some starting food
        self.board_state['food'] = list()
        for _ in range(5):
            self._populate_food()
        
    def _is_occupied(self, state: dict) -> np.ndarray:
        # only 1 channel is needed
        state_matrix = np.zeros((state["height"], state["width"]))

        # -- fill --
        for _snake in state['snakes']:
            for _body_segment in _snake['body']:
                state_matrix[_body_segment['x'], _body_segment['y']] = 1

        for _food in state['food']:
            state_matrix[_food['x'],_food['y']] = 1
        
        return state_matrix-1
    
    
    def _select_empty_loc(self) -> dict:
        # make an array of possible locations

        _temp_indexes = np.array(np.nonzero(self._is_occupied(self.board_state))).T
        # shuffle
        _temp_indexes = _temp_indexes[np.random.permutation(len(_temp_indexes))]
        return {'x':_temp_indexes[0][0], 'y':_temp_indexes[0][1]}
    
    def _populate_food(self):
        self.board_state['food'].append(self._select_empty_loc())
    
    def get_state(self) -> dict:
        # return a dictionary of the current state of the game
        # (following similar to battlensake dictionary)
        reduced_board_state = copy.deepcopy(self.board_state)

        for _snake in reduced_board_state['snakes']:
            del _snake['desired']

        return reduced_board_state
    
    def _move_target(self, current_pos: dict, move_dir: int) -> dict:
        if move_dir == 0: current_pos['y'] += 1 # up
        if move_dir == 1: current_pos['y'] -= 1 # down
        if move_dir == 2: current_pos['x'] -= 1 # left
        if move_dir == 3: current_pos['x'] += 1 # right
        return current_pos
        
    def step(self) -> str: # returns either 'running' or 'complete'
        
        # --------- save the current game state into history -------------
        _temp_dict = copy.deepcopy(self.board_state)
        self.history.append(_temp_dict)

        # ----------- request updates from living snakes -------------

        for _snake_class in self.snakes: # for each snake class
            for _snake in self.board_state['snakes']: # loop over snakes in board (to select the instance)
                if _snake['name'] == _snake_class.name:
                    
                    # request move from snake
                    _temp_move = _snake_class.move(self.get_state())
                    
                    # store the move in this hidden key
                    _snake['desired'] = self._move_target(_snake['head'].copy(), _temp_move) # absolute position
                    
                    # instantly remove a snake if it moves into a wall (should this be done later?)
                    if _snake['desired']['x'] > self.width or _snake['desired']['x'] < 0 or _snake['desired']['y'] > self.height or _snake['desired']['y'] < 0:
                        self.board_state['snakes'].remove(_snake)
                        print(_snake['name'],'went out of bounds')


        # ----------------- face-off -----------------
        pseudo_dead_snakes = list()
        
        # iterate over all pairs of snakes
        for _temp_snake in self.board_state['snakes']:
            for _temp_snake2 in self.board_state['snakes']:
                if _temp_snake['name'] != _temp_snake2['name']:
                    if _temp_snake['desired'] == _temp_snake2['desired']:
                        # collision, fight it out!
                        if _temp_snake['length'] <= _temp_snake2['length']:
                            pseudo_dead_snakes.append(_temp_snake['name'])

        # remove based on collision results
        for _pseudo_snake in pseudo_dead_snakes:
            for _snake in self.board_state['snakes']:
                if _pseudo_snake == _snake['name']:
                    self.board_state['snakes'].remove(_snake)
                    print(_snake['name'],'faced off and lost')

        # ------------ remove tail ---------------
        # is snake eating?
        eating_snakes = list()
        
        for _snake in self.board_state['snakes']:
            for _food in self.board_state['food']:
                if _snake['desired'] == _food: # comparing dictionaries
                    eating_snakes.append(_snake['name'])
                    self.board_state['food'].remove(_food)

        # shrink snake (via aging body segments)
        for _snake in self.board_state['snakes']:
            # append body
            if _snake['name'] not in eating_snakes:
                _snake['body'].pop()
            else:
                _snake['health'] = 100 # eating!
                _snake['length'] += 1
                
        # ---------------- obstacle array -----------------
        # note: out of bounds is covered above
        temp_array = np.zeros((self.height, self.width))
        # fill array with snake bodies
        for _snake in self.board_state['snakes']:
            for _body_segment in _snake['body']:
                # this removes out of bounds 
                tempx = np.clip(_body_segment['x'], 0, self.width-1)
                tempy = np.clip(_body_segment['y'], 0, self.height-1)
                temp_array[tempx][tempy] = 1

        # possibility of additional arbitrary obstacles
        # for _obstacle in self.board_state['obstacles']:
        #     temp_array[_obstacle['x']][_obstacle['y']] = 1
        
        # get all indexes of obstacles into a list of dicitonaries with the keys, 'x', and 'y'.
        _temp_indexes = np.array(np.nonzero(temp_array)).T
        _temp_obs_dict = list()
        for _row in _temp_indexes:
            _temp_obs_dict.append({'x':_row[0], 'y':_row[1]})
        
        # check if the snake is moving into an obstacle
        pesudo_dead_snakes = list()
        for _snake in self.board_state['snakes']:
            if _snake['desired'] in _temp_obs_dict:
                pesudo_dead_snakes.append(_snake)

        # remove based on collision results
        for _pseudo_snake in pseudo_dead_snakes:
            for _snake in self.board_state['snakes']:
                if _pseudo_snake == _snake['name']:
                    self.board_state['snakes'].remove(_snake)
                    print(_snake['name'],'runs into another snake at',_snake['desired'])
                    

        # ----- move head ------
        for _snake in self.board_state['snakes']:
            _snake['body'].insert(0,_snake['desired'])
            _snake['head'] = _snake['desired']

        
        # --------------- starve --------------
        # decrease health
        for _snake in self.board_state['snakes']:
            _snake['health'] -= 1
            if _temp_snake['health'] <= 0: # remove if starved
                self.board_state['snakes'].remove(_snake)  
                print(_snake['name'],'starved')

        # --------------- internal stuff -----------
        
        # ---- add food -----
        n_food = len(self.board_state['food'])
        if (n_food < self.rules['food_max']):
            if np.random.rand() < self.rules['food_rate']:
                self._populate_food()
        
        if n_food < self.rules['food_min']:
            self._populate_food()
        
        
        # --- check game end ---- 
        if self.rules['game_mode'] == 'duel': # multiple snakes, over when only 1 remains
            if len(self.board_state['snakes']) == 1:
                print(self.board_state['snakes'][0]['name'],'wins')
                _temp_dict = copy.deepcopy(self.board_state)
                self.history.append(_temp_dict)
                return 'complete'
            elif len(self.board_state['snakes']) == 0:
                print('draw')
                _temp_dict = copy.deepcopy(self.board_state)
                self.history.append(_temp_dict)
                return 'complete'
            else:
                return 'running'
        elif self.rules['game_mode'] == 'solo':
            if len(self.board_state['snakes']) == 0:
                _temp_dict = copy.deepcopy(self.board_state)
                self.history.append(_temp_dict)
                return 'complete'

            else: return 'running'

### initialize new game
define the snakes, give names. different snake classes can be made to follow different protocols


In [15]:
my_game_engine = game_engine()
my_game_engine.initialize(board = (11,11),
                          snakes = [simple_snake('tom'),
                                    simple_snake('molly'),
                                    simple_snake('sam')],
                          rules = rules)
                          

### play the game
and print out report

In [16]:
from time import perf_counter

start_time = perf_counter()
game_state = 'running'
n_turns = 0
#np.random.seed(17) # fixes the randomness for repeatability (for now)

while (game_state == 'running'): # should take less than a couple seconds on a slow computer
    game_state = my_game_engine.step()
    n_turns+=1
    
end_time = perf_counter()
elapsed_time = end_time-start_time
print(f'time: {elapsed_time:.4}s, n turns {n_turns},\n     ({n_turns/elapsed_time:.3f} tps, {elapsed_time/n_turns:.3f} spt)')



tom went out of bounds
sam went out of bounds
molly wins
time: 0.4693s, n turns 115,
     (245.051 tps, 0.004 spt)


### plot the match
(optional)

In [8]:
''' plot the match (if you want) '''
from matplotlib.colors import LinearSegmentedColormap

def checkerboard(shape):
    # from https://stackoverflow.com/questions/2169478/how-to-make-a-checkerboard-in-numpy
    return np.indices(shape).sum(axis=0) % 2

def plot_match(game_history, save_folder):

    # define colour space for background tiles
    cmap = LinearSegmentedColormap.from_list('mycmap', ['lightgrey', 'white'])

    # match snake colours to name, so that snake colours are constant when a snake is eliminated. 
    snake_cmaps = ['Greens','Blues','Purples','Oranges']
    snake_cmap_matched = {}
    for idx, _snake in enumerate(game_history[0]['snakes']): # starting snapshot
        snake_cmap_matched[_snake['name']] = snake_cmaps[idx]


    # loop through each step, save figure
    for idx, (_temp_history) in tqdm(enumerate(game_history)):

        # prepare board layers
        board_size = (_temp_history["height"], _temp_history["width"])

        #food
        food_layer = np.zeros(board_size)
        for _food in _temp_history['food']:
            food_layer[_food['x']][_food['y']] = 1

        # snakes
        snake_layers = {}
        for _snake in _temp_history['snakes']:
            temp_layer = np.zeros(board_size)
            shader = 0
            for _body in _snake['body']:
                temp_layer[_body['x']][_body['y']] = 1 + shader
                shader += 1
            snake_layers[_snake['name']] = temp_layer


        # ---------- start figure -----------
        plt.figure()

        # plot background
        plt.imshow(checkerboard(board_size),cmap=cmap)

        # plot food
        plt.scatter(*np.nonzero(food_layer),c='tab:orange',marker='s',s=100,edgecolors='k')

        # plot snakes (room for improvement here)
        for _snake in _temp_history['snakes']:

            plt.scatter(*np.nonzero(snake_layers[_snake['name']]),
                        c = snake_layers[_snake['name']][np.nonzero(snake_layers[_snake['name']])],
                        cmap = snake_cmap_matched[_snake['name']],
                        marker='s',
                        s=100,
                        edgecolors='k')

        plt.xticks([])
        plt.yticks([])
        plt.savefig(f'{save_folder}{idx:04}.png')
        plt.close()
        
        
# actually call this function
plot_match(game_history = my_game_engine.history,
           save_folder = '/Users/Finn/desktop/test3/')

110it [00:11,  9.84it/s]


#### and make it into a gif to post on discord

In [9]:
''' make gif from a folder '''
import imageio
from glob import glob

def make_gif(folder, save_path): 
    # load all .pngs in that folder as still images, then slap them together.

    if folder[-5:] != '*.png':
        folder += '*.png'
        
    # get paths to all files
    image_paths = glob(folder)
    image_paths.sort()

    # load each still
    stills_list=list()
    for _path in tqdm(image_paths):
        stills_list.append(imageio.imread(_path))

    # convert list of stills into a gif
    imageio.mimsave(save_path, stills_list, duration=0.1)
    print('gif saved')

make_gif(folder='/Users/Finn/desktop/test3/',
        save_path = '/Users/Finn/desktop/test3/*top.gif')

  0%|                                                   | 0/153 [00:00<?, ?it/s]/var/folders/tn/9tq725tx3bj46cw90w451p_r0000gp/T/ipykernel_97249/43198603.py:18: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  stills_list.append(imageio.imread(_path))
100%|████████████████████████████████████████| 153/153 [00:01<00:00, 109.97it/s]


gif saved


### save the game
as a json file

In [10]:
import json


class NumpyEncoder(json.JSONEncoder):
    """ Custom encoder for numpy data types """
    # json can't serialize numpy ints, so:
    # from https://stackoverflow.com/questions/50916422/python-typeerror-object-of-type-int64-is-not-json-serializable
    # " hmallen's code in numpyencoder/numpyencoder.py "
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):

            return int(obj)

# get last saved game name
paths = glob('/Users/Finn/desktop/battlesnake_games/bs_game*.json')
paths.sort()

if len(paths) < 1: game_name = 'bs_game0'
else: game_name = 'bs_game' + str(len(paths))
    
with open(f'/Users/Finn/desktop/battlesnake_games/{game_name}.json','w') as game_file:
    json.dump(my_game_engine.history, game_file,cls=NumpyEncoder)

### replay part of a game

In [11]:

''' preserve the actual game history ''' # or could load from a saved file
actual_game_history = copy.deepcopy(my_game_engine.history)


In [12]:

''' example of using some game state (dictionary) to initialize '''

# copy some board history 
hallucinated_game = game_engine()
hallucinated_game.init_on_game_state(game_state = actual_game_history[-6], # go back 6 turns from the end
                                     rules=rules)


In [13]:
from time import perf_counter

start_time = perf_counter()
game_state = 'running'
n_turns = 0

while (game_state == 'running'):
    n_turns+=1
    game_state = hallucinated_game.step()

end_time = perf_counter()
elapsed_time = end_time-start_time
print(f'time: {elapsed_time:.4}s, n turns {n_turns},\n     ({n_turns/elapsed_time:.3f}tps, {elapsed_time/n_turns:.3f}spt)')



tom runs into another snake at {'x': 10, 'y': 6}
sam wins
time: 0.04487s, n turns 8,
     (178.283tps, 0.006spt)
